In [35]:
import json
import pandas as pd
import jieba
import pickle
import os
from collections import Counter
from pypinyin import pinyin, Style
from deep_translator import GoogleTranslator
from deep_translator.exceptions import TranslationNotFound
import re
from tqdm import tqdm


In [3]:
# 1. Load Data
with open('../data/raw/z_channel_join_new_company.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
full_text = " ".join([d['text'] for d in data])

In [4]:
# 2. Load Custom Malaysian Dictionary
# Ensure you have 'assets/msia_dict.txt' with terms like RTS and CIQ
if os.path.exists('../assets/msia_dict.txt'):
    jieba.load_userdict('../assets/msia_dict.txt')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\User\AppData\Local\Temp\jieba.cache
Loading model cost 0.787 seconds.
Prefix dict has been built successfully.


In [5]:
# 3. Tokenize & Filter
# Remove stop words and single-character words (grammar particles)
stop_words = {'這個', '這邊', '就是', '一個', '可以', '他們', '因為', '現在', '如果', '一些', '什麼', '所以'}
tokens = [t for t in jieba.lcut(full_text) if len(t) > 1 and t not in stop_words]

In [6]:
# 4. Save for PCA (Notebook 03)
os.makedirs('../data/processed', exist_ok=True)
with open('../data/processed/clean_tokens.pkl', 'wb') as f:
    pickle.dump(tokens, f)

In [7]:
# 5. EDA: Top 10 Phrases
print("Top 10 Phrases:")
print(pd.DataFrame(Counter(tokens).most_common(10), columns=['Word', 'Count']))

Top 10 Phrases:
  Word  Count
0   其實     25
1   工作     25
2   自己     25
3   西亞     19
4   可能     19
5   分享     19
6   東西     18
7   時候     17
8   那個     17
9  新加坡     16


## For studying

In [36]:
def get_pinyin(text):
    # Style.TONE provides marks like ā, á, ǎ, à
    pinyin_list = pinyin(text, style=Style.TONE)
    return " ".join([item[0] for item in pinyin_list])

def count_mandarin(text):
    # This regex looks for the Unicode range of all Chinese characters
    mandarin_only = re.findall(r'[\u4e00-\u9fff]', text)
    return len(mandarin_only)

def to_seconds(time_str):
    m, s = map(int, time_str.split(':'))
    return m * 60 + s

def contains_chinese(text):
    return bool(re.search(r'[\u4e00-\u9fff]', text))

def safe_translate_zh_cn(text):
    try:
        translated = translator.translate(text)
        
        # Skip if translation still contains Chinese
        if contains_chinese(translated):
            return None
        
        return translated
    
    except TranslationNotFound:
        return None
    except Exception:
        return None

In [33]:
df = pd.DataFrame(data)

In [34]:
df['mandarin_length'] = df['text'].apply(count_mandarin)
df = df.sort_values(by='mandarin_length', ascending=False)
df = df[df['mandarin_length'] > 0]

In [20]:
# 4. Select the top 30 longest sentences to study
# These are likely to be full explanations or descriptions
study_df = df.head(30).copy()

print(f"⏳ Processing the 30 longest sentences for your study guide...")

⏳ Processing the 30 longest sentences for your study guide...


In [ ]:
# Generate Pinyin
study_df['pinyin'] = study_df['text'].apply(get_pinyin)

# Generate English Translation
translator = GoogleTranslator(source='zh-CN', target='en')
study_df['english'] = study_df['text'].apply(lambda x: translator.translate(x))

In [30]:
study_df

,text,start_raw,start_formatted,duration_raw,duration_formatted,mandarin_length,pinyin,english
251,就想說這個可能就是一個機會讓我去試一試不一樣的東西,458.800,00:07:38:800,3.833,00:00:03:833,25,jiù xiǎng shuō zhè gè kě néng jiù shì yī gè jī...,I just want to say that this might be an oppor...
418,你知道現在新加坡對馬來西亞的匯率是多少嗎,762.166,00:12:42:166,2.800,00:00:02:799,20,nǐ zhī dào xiàn zài xīn jiā pō duì mǎ lái xī y...,Do you know what the exchange rate between Sin...
171,怎麼樣講身為一個爸爸都一定要負起一點責任,321.633,00:05:21:632,3.167,00:00:03:166,20,zěn me yàng jiǎng shēn wèi yī gè bà bà dōu yí ...,"How do you say that as a father, you must take..."
264,其實我沒有在講這個前公司的壞話還是什麼啊,489.600,00:08:09:600,3.300,00:00:03:299,20,qí shí wǒ méi yǒu zài jiǎng zhè gè qián gōng s...,"Actually, I'm not talking bad about this forme..."
111,我覺得這個就是我要呈現的內容的藝術啦,220.200,00:03:40:199,3.500,00:00:03:500,18,wǒ jué dé zhè gè jiù shì wǒ yào chéng xiàn de ...,I think this is the art of the content I want ...
230,是跟我之前做內容創作的東西是很相似的,422.066,00:07:02:065,2.867,00:00:02:867,18,shì gēn wǒ zhī qián zuò nèi róng chuàng zuò de...,It’s very similar to what I used to do in cont...
260,又或者是你自己想要的工作環境的文化哦,478.200,00:07:58:199,4.633,00:00:04:633,18,yòu huò zhě shì nǐ zì jǐ xiǎng yào de gōng zuò...,Or maybe it’s the culture of the work environm...
303,然後你是沒有辦法可以省略掉這些東西的,562.466,00:09:22:466,2.467,00:00:02:467,18,rán hòu nǐ shì méi yǒu bàn fǎ kě yǐ shěng lüè ...,Then there is no way you can omit these things
232,我會選它的原因是因為它非常靠近我的家,426.700,00:07:06:699,3.666,00:00:03:665,18,wǒ huì xuǎn tā de yuán yīn shì yīn wèi tā fēi ...,The reason I chose it is because it's very clo...
24,我也不知道這支影片會在什麼時候播出,50.866,00:00:50:865,2.634,00:00:02:633,17,wǒ yě bù zhī dào zhè zhī yǐng piàn huì zài shé...,I don’t know when this video will be released


In [31]:
OUTPUT_FILE = '../data/processed/join_new_company_long_sentences_study.csv'
# Adding 'length' to the output so you can see how "big" the sentence is
final_columns = ['start_formatted', 'mandarin_length', 'text', 'pinyin', 'english']
study_df[final_columns].to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

### By Contextual Clip

In [24]:
START_TIME = "01:41"  # 1 minute
END_TIME   = "14:28"  # 2 minutes

In [25]:
start_s = to_seconds(START_TIME)
end_s   = to_seconds(END_TIME)

In [37]:
study_df_context = df[(df['start_raw'] >= start_s) & (df['start_raw'] <= end_s)].copy()
study_df_context = study_df_context.sort_values(by='start_raw')

print(f"🎬 Found {len(study_df_context)} lines between {START_TIME} and {END_TIME}")

🎬 Found 417 lines between 01:41 and 14:28


In [38]:
tqdm.pandas()

study_df_context['pinyin'] = study_df_context['text'].apply(get_pinyin)
# study_df_context['english'] = study_df_context['text'].progress_apply(lambda x: translator.translate(x))
study_df_context['english'] = (study_df_context['text'].progress_apply(safe_translate_zh_cn))

100%|██████████| 417/417 [07:22<00:00,  1.06s/it]


In [39]:
output_context = f'../data/processed/join_new_company_context_{START_TIME.replace(":","-")}.csv'
study_df_context[['start_formatted', 'text', 'pinyin', 'english']].to_csv(output_context, index=False, encoding='utf-8-sig')

print(f"✅ Clip study guide saved: {output_context}")

✅ Clip study guide saved: ../data/processed/join_new_company_context_01-41.csv
